In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
import pandas as pd
import time
import random
import tqdm
import pyautogui
from bs4 import BeautifulSoup
import requests
from selenium.webdriver.common.by import By
import time 
import datetime

In [2]:
browser = webdriver.Chrome()

In [3]:
df = pd.DataFrame(columns = ['Что ищем','Время сбора','Ссылка','name','adr','work','ost','otz','osob','inf'])

In [4]:
browser.get(r'https://yandex.ru/maps/')

In [6]:
Poisk = browser.find_element(By.CLASS_NAME,'input__control')
adress = pd.read_excel('1500-2000.xlsx')
adress = list(adress['адрес'])
adress

['Респ Татарстан г Набережные Челны Гидростроителей 13  ',
 'Респ Татарстан г Набережные Челны Набережночелнинский 70/56  ',
 'Респ Татарстан г Нижнекамск Кайманова 16  ',
 'обл Московская г Серпухов Бригадная 7  ',
 'обл Псковская г Невель Ленина 34  ',
 'г Москва г Москва Шарикоподшипниковская 13стр1  ',
 'обл Саратовская г Энгельс Марины Расковой 12  ',
 'обл Саратовская г Саратов Им Навашина С.г. 8  ',
 'обл Саратовская г Саратов им Сакко и Ванцетти 15  ',
 'край Пермский г Пермь Ласьвинская 37  ',
 'обл Рязанская  Мосолово Рощина 32  ',
 'край Пермский г Чайковский Вокзальная 2  ',
 'край Хабаровский г Хабаровск Морозова Павла Леонтьевича 86  ',
 'край Хабаровский г Хабаровск 60-летия Октября 152  ',
 'край Хабаровский г Комсомольск-на-Амуре Дзержинского 46 корпус 3',
 'край Хабаровский г Комсомольск-на-Амуре Красноармейская 18  ',
 'край Краснодарский г Гулькевичи Советская 14  ',
 'край Краснодарский г Краснодар МИРНЫЙ 16  ',
 'обл Тамбовская  Токаревка Ленина 3  ',
 'край Пермс

In [7]:
Poisk = browser.find_element(By.CLASS_NAME,'input__control')

In [ ]:
def sbor(aa):
    temp = pd.DataFrame(columns = ['Что ищем','Время сбора','Ссылка','name','adr','work','ost','otz','osob','inf'])
    time.sleep(1)
    
    work = ''
    
    if browser.find_elements(By.CLASS_NAME,'card-feature-view__value')[0].text == 'Добавить':
        work = browser.find_elements(By.CLASS_NAME,'business-working-status-view')[-1].text
    elif browser.find_elements(By.CLASS_NAME,'card-feature-view__value')[0].text == 'График':
        browser.find_elements(By.CLASS_NAME,'card-feature-view__value')[0].click()
        time.sleep(1)
        #browser.find_elements(By.CLASS_NAME,'business-working-status-view')[0].click()
        time.sleep(2)
        e = browser.find_elements(By.CLASS_NAME,'business-working-intervals-view__item')
        
        for ee in e:
            work = work + ' ' + ee.text
        time.sleep(1)
    
    name = browser.find_elements(By.CLASS_NAME,'card-title-view__title-link')
    name = name[0].text
    
    adr = browser.find_elements(By.CLASS_NAME,'business-contacts-view__address-link')
    adr = adr[0].text

    try:
        ost = browser.find_element(By.CLASS_NAME,'masstransit-stops-view__stop-name').text
        metrs = browser.find_element(By.CLASS_NAME,'masstransit-stops-view__stop-distance').text
        ost_info = ost + ' ' + metrs
    except:
        ost_info = None
        

    features = browser.find_elements(By.CLASS_NAME,'business-features-view__bool-text')

    vkl = browser.find_elements(By.CLASS_NAME,'tabs-select-view__title')
    
    try:
        vkl[2].click()
        time.sleep(5)
    except:
        temp = temp.append({'Что ищем':aa,'Время сбора':datetime.datetime.now(),'Ссылка':'','name':'','adr':'','work':'','ost':'','otz':'','osob':'','inf':''},ignore_index=True)
        return temp

    o = browser.find_elements(By.CLASS_NAME,'business-review-view__body-text')
    otz = []
    for i in range(len(o)):
        otz.append(o[i].text)
    
    time.sleep(2)
    vkl[4].click()
    time.sleep(5)

    o = browser.find_elements(By.CLASS_NAME,'business-features-view__bool-text')
    osob = []
    for i in range(len(o)):
        osob.append(o[i].text)

    time.sleep(3)

    o = browser.find_elements(By.CLASS_NAME,'business-features-view__valued-value')
    oper = []
    for i in range(len(o)):
        oper.append(o[i].text)
    temp = temp.append({'Что ищем':aa,'Время сбора':datetime.datetime.now(),'Ссылка':browser.current_url,'name':name,'adr':adr,'work':work,'ost':ost_info,'otz':otz,'osob':osob,'inf':oper},ignore_index=True)
    return temp

c = 0
for aa in adress:
    if c<0:
        pass
    else:
        Poisk.send_keys(Keys.CONTROL + 'a')
        Poisk.send_keys(Keys.DELETE)
        time.sleep(1)


        Poisk.send_keys('сбер банкомат ' + aa.replace('отделение Сбербанка','').replace('.',' ').replace(',',' '))
        time.sleep(2.5)
        Poisk.send_keys(Keys.ENTER)
        time.sleep(5)
        h = ''
        elem = browser.find_elements(By.CLASS_NAME,'search-business-snippet-view')
        if len(elem)>0:
            #if len(elem)<10:
            if len(browser.find_elements(By.CLASS_NAME,'card-section-header__title'))>0:
                df = df.append(pd.Series(), ignore_index=True)
                df['Что ищем'][len(df)-1] = aa
            else:
                elem[0].click()
                df = df.append(sbor(aa))

        else:
            if len(browser.find_elements(By.CLASS_NAME,'tabs-select-view__title'))<1:
                print(1)
                df = df.append(pd.Series(), ignore_index=True)
                df['Что ищем'][len(df)-1] = aa
            else:
                if len(browser.find_elements(By.CLASS_NAME,'nothing-found-view__header'))>0:
                    print(12)
                    df = df.append(pd.Series(), ignore_index=True)
                    df['Что ищем'][len(df)-1] = aa
                else:
                    print(10)
                    df = df.append(sbor(aa))
        

    c = c + 1       

10


C:\Users\User1\AppData\Local\Temp\ipykernel_3676\4055140824.py:65: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  temp = temp.append({'Что ищем':aa,'Время сбора':datetime.datetime.now(),'Ссылка':browser.current_url,'name':name,'adr':adr,'work':work,'ost':ost_info,'otz':otz,'osob':osob,'inf':oper},ignore_index=True)
C:\Users\User1\AppData\Local\Temp\ipykernel_3676\4055140824.py:105: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(sbor(aa))
C:\Users\User1\AppData\Local\Temp\ipykernel_3676\4055140824.py:65: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  temp = temp.append({'Что ищем':aa,'Время сбора':datetime.datetime.now(),'Ссылка':browser.current_url,'name':name,'adr':adr,'work':work,'ost':ost_info,'otz':otz,'osob

In [173]:
df.to_excel('Yandex 920-1000.xlsx',index=False)

In [164]:
c

920

In [107]:
len(browser.find_elements(By.CLASS_NAME,'tabs-select-view__title'))<1

False

In [118]:
vkl = browser.find_elements(By.CLASS_NAME,'tabs-select-view__title')

In [64]:
'отделение Сбербанка ' + adress[4].replace('отделение Сбербанка','').replace('.',' ').replace(',',' ')

'отделение Сбербанка Юрлинский район  с Усть-Зула  ул Центральная  41  '

In [185]:
df

,Что ищем,Время сбора,Ссылка,name,adr,work,ost,otz,osob,inf
0,"175000, обл Новгородская, р-н Батецкий, , п Б...",2024-03-21 11:30:26.723340,https://yandex.ru/maps/org/sberbank/2634055884...,Сбербанк,"Первомайская ул., 27, посёлок Батецкий",Понедельник\nКруглосуточно Вторник\nКруглосут...,Станция Батецкая 600 м,[],"[, Внесение наличных, Бесконтактные технологии...","[свободный, 30.09.2023, RUB, RUB, выдача налич..."
0,"175000, обл Новгородская, р-н Батецкий, , п Б...",2024-03-21 11:30:48.838089,https://yandex.ru/maps/org/sberbank/1030479122...,СберБанк,"Первомайская ул., 27, посёлок Батецкий",Понедельник\n08:30 – 12:0013:00 – 16:15 Вторн...,Станция Батецкая 610 м,"[Вежливый компетентный персонал , нет очереди ...","[, , , Данные Банка России, Обслуживание физич...","[1481/960/33, 11.03.2024, дополнительный офис,..."


In [189]:
df = df.reset_index()

In [191]:
df['inf'][1]

['1481/960/33',
 '11.03.2024',
 'дополнительный офис',
 'брокер, дилер, депозитарий, инвестиционный советник',
 '1481',
 'вклады в рублях, вклады в валюте, вклады в драгоценных металлах, операции с безналичной валютой, операции с наличной валютой, операции с драгоценными металлами',
 'действующая',
 'Sberbank of Russia, Sberbank',
 'Публичное акционерное общество "Сбербанк России"',
 'ПАО Сбербанк',
 '7707083893',
 '1027700132195',
 'для корпоративных клиентов, для предпринимателей, среднего и малого бизнеса, для частных лиц',
 'потребительский кредит']